In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!unzip '/content/drive/MyDrive/Kaggle/data.zip' -d '/content/'

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
from cv2 import imread, createCLAHE # read and equalize images
from glob import glob
import matplotlib.pyplot as plt
%matplotlib inline
from pandas.core.common import flatten

import os
import torch
import pandas as pd
from skimage import io, transform
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils

# Ignore warnings
import warnings
warnings.filterwarnings("ignore")

plt.ion()   # interactive mode

In [ ]:
from __future__ import print_function, division

# pytorch imports
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
from torch.autograd import Variable
import torchvision
from torchvision import datasets, models, transforms
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils

# image imports
from skimage import io, transform
from PIL import Image

# general imports
import os
import time
from shutil import copyfile
from shutil import rmtree

# data science imports
import pandas as pd
import numpy as np
import csv

In [ ]:
!pip install lightly

In [ ]:
from itertools import chain
import random
np.random.seed(25)

df = pd.read_csv('/content/Data_Entry_2017.csv')
df.drop(['OriginalImage[Width', 'Height]', 'OriginalImagePixelSpacing[x', 'y]', 'Unnamed: 11'], axis=1, inplace=True)
num_obs = len(df)
# print('Number of observations:',num_obs)

my_glob = glob('/content/images*/images/*.png')
# print('Number of Observations: ', len(my_glob))

full_img_paths = {os.path.basename(x): x for x in my_glob}
df['full_path'] = df['Image Index'].map(full_img_paths.get)

train_val_list = pd.read_csv('/content/train_val_list.txt', header=None, names = ['image_list'])
test_list = pd.read_csv('/content/test_list.txt', header=None, names = ['image_list'])

train = df[df['Image Index'].isin(train_val_list['image_list'].values)].reset_index(drop=True)
test = df[df['Image Index'].isin(test_list['image_list'].values)].reset_index(drop=True)

labels_discard = ['Consolidation', 'Edema', 'Emphysema', 'Fibrosis', 'Hernia', 'Pleural_Thickening']
for i in labels_discard:
    train = train[~train['Finding Labels'].str.contains(i)]
    test = test[~test['Finding Labels'].str.contains(i)]

# train = pd.concat([train[~train['Finding Labels'].str.contains('No Finding')],
#                   train[train['Finding Labels'].str.contains('No Finding')].drop_duplicates(subset=['Finding Labels', 'Patient ID', 'View Position'], keep='first')]).sort_values(by='Image Index')

def one_hot_enc(df):
    df['Finding Labels'] = df['Finding Labels'].map(lambda x: x.replace('No Finding', ''))
    all_labels = np.unique(list(chain(*df['Finding Labels'].map(lambda x: x.split('|')).tolist())))
    for c_label in all_labels:
        if len(c_label)>1: # leave out empty labels
            df[c_label] = df['Finding Labels'].map(lambda finding: 1 if c_label in finding else 0)
    return df

train = one_hot_enc(train)
test = one_hot_enc(test)

train_image_paths = list(flatten(train['full_path'].values))
test_image_paths = list(flatten(test['full_path'].values))

l = np.unique(train['Patient ID'].values)
np.random.shuffle(l)
cut = int(np.round(((90/100)*len(l)), decimals=0))
train_values = l[:cut]
val_values = l[cut:]
train_dict = dict.fromkeys(train_values, 'train')
train_dict.update(dict.fromkeys(val_values, 'val'))
train['fold'] = train['Patient ID'].map(train_dict.get)

train = train[['full_path', 'Image Index', 'fold', 'Atelectasis', 'Cardiomegaly', 'Effusion', 'Infiltration',
               'Mass', 'Nodule', 'Pneumonia', 'Pneumothorax']]
test['fold'] = 'test'
test = test[['full_path', 'Image Index', 'fold', 'Atelectasis', 'Cardiomegaly', 'Effusion', 'Infiltration',
             'Mass', 'Nodule', 'Pneumonia', 'Pneumothorax']]
train.reset_index(drop=True, inplace=True)
test.reset_index(drop=True, inplace=True)

In [ ]:
import torch
import torch.nn as nn
import torchvision

from torch.utils.data import (
    Dataset,
    DataLoader,
)

from torchvision.datasets import ImageFolder
from torchvision.transforms import (
    RandomResizedCrop,
    RandomHorizontalFlip,
    ColorJitter,
    RandomGrayscale,
    RandomApply,
    Compose,
    GaussianBlur,
    ToTensor,
)
import torchvision.models as models

import os
import glob
import time
from skimage import io
import matplotlib.pyplot as plt

In [ ]:
from PIL import Image as img

class CreateDataset(Dataset):

    def __init__(self, train, fold, transform):
        self.df = train
        self.transform = transform
        self.df = self.df[self.df['fold'] == fold]

        self.df = self.df.set_index("Image Index")
        self.PRED_LABEL = ['Atelectasis',
                           'Cardiomegaly',
                           'Effusion',
                           'Infiltration',
                           'Mass',
                           'Nodule',
                           'Pneumonia',
                           'Pneumothorax']
        RESULT_PATH = "results/"

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()

        image = img.open(self.df.iloc[idx, 0])
        image = image.convert('RGB')

        label = np.zeros(len(self.PRED_LABEL), dtype=int)
        for i in range(0, len(self.PRED_LABEL)):
            if(self.df[self.PRED_LABEL[i].strip()].iloc[idx].astype('int') > 0):
                label[i] = self.df[self.PRED_LABEL[i].strip()
                                  ].iloc[idx].astype('int')

        if self.transform:
            image = self.transform(image)

        return image

In [ ]:
import torch
import torchvision
from torch import nn

from lightly.loss import NegativeCosineSimilarity
from lightly.models.modules import SimSiamPredictionHead, SimSiamProjectionHead
from lightly.transforms import FastSiamTransform

In [ ]:
from typing import Optional, Tuple, Union

import torchvision.transforms as T
from PIL.Image import Image
from torch import Tensor

from lightly.transforms.gaussian_blur import GaussianBlur
from lightly.transforms.multi_view_transform import MultiViewTransform
from lightly.transforms.rotation import random_rotation_transform
from lightly.transforms.utils import IMAGENET_NORMALIZE


class SimSiamTransform(MultiViewTransform):
    """Implements the transformations for SimSiam.

    Input to this transform:
        PIL Image or Tensor.

    Output of this transform:
        List of Tensor of length 2.

    Applies the following augmentations by default:
        - Random resized crop
        - Random horizontal flip
        - Color jitter
        - Random gray scale
        - Gaussian blur
        - ImageNet normalization

    Attributes:
        input_size:
            Size of the input image in pixels.
        cj_prob:
            Probability that color jitter is applied.
        cj_strength:
            Strength of the color jitter. `cj_bright`, `cj_contrast`, `cj_sat`, and
            `cj_hue` are multiplied by this value. For datasets with small images,
            such as CIFAR, it is recommended to set `cj_strength` to 0.5.
        cj_bright:
            How much to jitter brightness.
        cj_contrast:
            How much to jitter constrast.
        cj_sat:
            How much to jitter saturation.
        cj_hue:
            How much to jitter hue.
        min_scale:
            Minimum size of the randomized crop relative to the input_size.
        random_gray_scale:
            Probability of conversion to grayscale.
        gaussian_blur:
            Probability of Gaussian blur.
        kernel_size:
            Will be deprecated in favor of `sigmas` argument. If set, the old behavior applies and `sigmas` is ignored.
            Used to calculate sigma of gaussian blur with kernel_size * input_size.
        sigmas:
            Tuple of min and max value from which the std of the gaussian kernel is sampled.
            Is ignored if `kernel_size` is set.
        vf_prob:
            Probability that vertical flip is applied.
        hf_prob:
            Probability that horizontal flip is applied.
        rr_prob:
            Probability that random rotation is applied.
        rr_degrees:
            Range of degrees to select from for random rotation. If rr_degrees is None,
            images are rotated by 90 degrees. If rr_degrees is a (min, max) tuple,
            images are rotated by a random angle in [min, max]. If rr_degrees is a
            single number, images are rotated by a random angle in
            [-rr_degrees, +rr_degrees]. All rotations are counter-clockwise.
        normalize:
            Dictionary with 'mean' and 'std' for torchvision.transforms.Normalize.

    """

    def __init__(
        self,
        input_size: int = 224,
        cj_prob: float = 0.8,
        cj_strength: float = 1.0,
        cj_bright: float = 0.4,
        cj_contrast: float = 0.4,
        cj_sat: float = 0.4,
        cj_hue: float = 0.1,
        min_scale: float = 0.2,
        random_gray_scale: float = 0.2,
#         gaussian_blur: float = 0.5,
        kernel_size: Optional[float] = None,
        sigmas: Tuple[float, float] = (0.1, 2),
        vf_prob: float = 0.0,
        hf_prob: float = 1.0,
        rr_prob: float = 1.0,
        rr_degrees: Union[None, float, Tuple[float, float]] = 10,
        normalize: Union[None, dict] = IMAGENET_NORMALIZE,
    ):
        view_transform = SimSiamViewTransform(
            input_size=input_size,
            cj_prob=cj_prob,
            cj_strength=cj_strength,
            cj_bright=cj_bright,
            cj_contrast=cj_contrast,
            cj_sat=cj_sat,
            cj_hue=cj_hue,
            min_scale=min_scale,
            random_gray_scale=random_gray_scale,
#             gaussian_blur=gaussian_blur,
            kernel_size=kernel_size,
            sigmas=sigmas,
            vf_prob=vf_prob,
            hf_prob=hf_prob,
            rr_prob=rr_prob,
            rr_degrees=rr_degrees,
            normalize=normalize,
        )
        super().__init__(transforms=[view_transform, view_transform])


class SimSiamViewTransform:
    def __init__(
        self,
        input_size: int = 224,
        cj_prob: float = 0.8,
        cj_strength: float = 1.0,
        cj_bright: float = 0.4,
        cj_contrast: float = 0.4,
        cj_sat: float = 0.4,
        cj_hue: float = 0.1,
        min_scale: float = 0.2,
        random_gray_scale: float = 0.2,
#         gaussian_blur: float = 0.5,
        kernel_size: Optional[float] = None,
        sigmas: Tuple[float, float] = (0.1, 2),
        vf_prob: float = 0.0,
        hf_prob: float = 1.0,
        rr_prob: float = 1.0,
        rr_degrees: Union[None, float, Tuple[float, float]] = 10,
        normalize: Union[None, dict] = IMAGENET_NORMALIZE,
    ):
        color_jitter = T.ColorJitter(
            brightness=cj_strength * cj_bright,
            contrast=cj_strength * cj_contrast,
            saturation=cj_strength * cj_sat,
            hue=cj_strength * cj_hue,
        )

        transform = [
            T.Resize(224),
            # T.CenterCrop(224),
#             T.RandomResizedCrop(size=input_size, scale=(min_scale, 0.8)),
            random_rotation_transform(rr_prob=rr_prob, rr_degrees=rr_degrees),
            T.RandomHorizontalFlip(p=hf_prob),
#             T.RandomVerticalFlip(p=vf_prob),
#             T.RandomApply([color_jitter], p=cj_prob),
#             T.RandomGrayscale(p=random_gray_scale),
#             GaussianBlur(kernel_size=kernel_size, sigmas=sigmas, prob=gaussian_blur),
            T.ToTensor(),
        ]
        if normalize:
            transform += [T.Normalize(mean=normalize["mean"], std=normalize["std"])]
        self.transform = T.Compose(transform)

    def __call__(self, image: Union[Tensor, Image]) -> Tensor:
        """
        Applies the transforms to the input image.

        Args:
            image:
                The input image to apply the transforms to.

        Returns:
            The transformed image.

        """
        return self.transform(image)

In [ ]:
transform = SimSiamTransform()

dataset = CreateDataset(train, 'train', transform=transform)

dataloader = torch.utils.data.DataLoader(
    dataset,
    batch_size=64,
    shuffle=True,
    drop_last=True,
    num_workers=8,
    pin_memory=True
)

In [ ]:
import torch
import torchvision
from torch import nn

from lightly.loss import NegativeCosineSimilarity
from lightly.models.modules import SimSiamPredictionHead, SimSiamProjectionHead
# from lightly.transforms import SimSiamTransform


class SimSiam_Train(nn.Module):
    def __init__(self, backbone):
        super().__init__()
        self.backbone = backbone
        self.projection_head = SimSiamProjectionHead(2048, 512, 128)
        self.prediction_head = SimSiamPredictionHead(128, 64, 128)

    def forward(self, x):
        f = self.backbone(x).flatten(start_dim=1)
        z = self.projection_head(f)
        p = self.prediction_head(z)
        z = z.detach()
        return z, p

In [ ]:
# Copyright (c) Facebook, Inc. and its affiliates.
# All rights reserved.

# This source code is licensed under the license found in the
# LICENSE file in the root directory of this source tree.

import torch
import torch.nn as nn


class SimSiam(nn.Module):
    """
    Build a SimSiam model.
    """
    def __init__(self, base_encoder, dim=2048, pred_dim=512):
        """
        dim: feature dimension (default: 2048)
        pred_dim: hidden dimension of the predictor (default: 512)
        """
        super(SimSiam, self).__init__()

        # create the encoder
        # num_classes is the output fc dimension, zero-initialize last BNs
        self.encoder = base_encoder(num_classes=dim, zero_init_residual=True)

        # build a 3-layer projector
        prev_dim = self.encoder.fc.weight.shape[1]
        self.encoder.fc = nn.Sequential(nn.Linear(prev_dim, prev_dim, bias=False),
                                        nn.BatchNorm1d(prev_dim),
                                        nn.ReLU(inplace=True), # first layer
                                        nn.Linear(prev_dim, prev_dim, bias=False),
                                        nn.BatchNorm1d(prev_dim),
                                        nn.ReLU(inplace=True), # second layer
                                        self.encoder.fc,
                                        nn.BatchNorm1d(dim, affine=False)) # output layer
        self.encoder.fc[6].bias.requires_grad = False # hack: not use bias as it is followed by BN

        # build a 2-layer predictor
        self.predictor = nn.Sequential(nn.Linear(dim, pred_dim, bias=False),
                                        nn.BatchNorm1d(pred_dim),
                                        nn.ReLU(inplace=True), # hidden layer
                                        nn.Linear(pred_dim, dim)) # output layer

    def forward(self, x1, x2):
        """
        Input:
            x1: first views of images
            x2: second views of images
        Output:
            p1, p2, z1, z2: predictors and targets of the network
            See Sec. 3 of https://arxiv.org/abs/2011.10566 for detailed notations
        """

        # compute features for one view
        z1 = self.encoder(x1) # NxC
        z2 = self.encoder(x2) # NxC

        p1 = self.predictor(z1) # NxC
        p2 = self.predictor(z2) # NxC

        return p1, p2, z1.detach(), z2.detach()

In [ ]:
model = SimSiam(
    models.__dict__['resnet50'],
    2048, 512)

model = nn.DataParallel(model)

# ckpt = torch.load('/kaggle/input/chestx-ray/fastsiam_2_resnet50_backbonemodel.pth')
# backbone.load_state_dict(ckpt['resnet50_parameters'])

checkpoint = torch.load('/content/drive/MyDrive/simsiam/checkpoint_0099.pth.tar')
model.load_state_dict(checkpoint['state_dict'])

backbone = nn.Sequential(*list(model.module.children())[:-1])
backbone = nn.Sequential(*list(backbone[0].children())[:-1])

model = SimSiam_Train(backbone)

# checkpoint = torch.load('/content/pretrained_simsiam_28_withprojectionhead.pth')
# model.load_state_dict(checkpoint)

device = "cuda" if torch.cuda.is_available() else "cpu"
model = model.to(device)
# model.load_state_dict(torch.load('/kaggle/input/chestx-ray/fastsiam_16_resnet50_withprojectionhead.pth'))
# model.to(device)
print('Model ready for training')

In [ ]:
model

In [ ]:
# for batch in dataloader:
#     x0, x1 = batch
#     break

In [ ]:
# images = next(iter(dataloader))

In [ ]:
# plt.figure()

# f, axarr = plt.subplots(2,1)

# axarr[0].imshow(images[1][16][0])
# axarr[1].imshow(images[0][16][1])

In [ ]:
criterion = NegativeCosineSimilarity()
optimizer = torch.optim.SGD(model.parameters(), lr=0.06, momentum=0.9, weight_decay=0.0001)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, 40)
loss_val_list = []

print("Starting Training")
for epoch in range(40):
    t0 = time.time()
    total_loss = 0
    for batch in dataloader:
        x0, x1 = batch
        x0 = x0.to(device)
        x1 = x1.to(device)
        z0, p0 = model(x0)
        z1, p1 = model(x1)
        loss = 0.5 * (criterion(z0, p1) + criterion(z1, p0))
        total_loss += loss.detach()
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
    scheduler.step()
    avg_loss = total_loss / len(dataloader)

    if ((epoch+1) == 1):
      model_save_name = f'simsiam_chestxray_{epoch+1}_withprojectionhead.pth'
      path = f'/content/drive/MyDrive/simsiam/{model_save_name}'
      torch.save(model.state_dict(), path)

    if ((epoch+1) % 4 == 0):
      model_save_name = f'simsiam_chestxray_{epoch+1}_withprojectionhead.pth'
      path = f'/content/drive/MyDrive/simsiam/{model_save_name}'
      torch.save(model.state_dict(), path)

    if ((epoch+1) % 2 == 0):
      torch.save(model.state_dict(), f'pretrained_simsiam_{epoch+1}_withprojectionhead.pth')
      state_dict = {'resnet50_parameters': model.backbone.state_dict()}
      torch.save(state_dict, f'pretrained_simsiam_{epoch+1}_backbonemodel.pth')
    print(f"epoch: {epoch+1:>02}, loss: {avg_loss:.5f}. Time taken: {((time.time()-t0)/60):.3f} mins")
    loss_val_list.append(avg_loss)

In [ ]:
for i in range(len(loss_val_list)):
  loss_val_list[i] = loss_val_list[i].item()
np.save('/content/simsiam_loss_40.npy', loss_val_list)